### Libs

In [2]:
from Utils.Containers import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Directories

In [3]:
# One folder containing series
_dir_dicom1 = r"C:\Users\CH258598\Desktop\Current Research\case-100012\omi\incomingdir\case-100012\STANDARD_HEAD-NECK-U-EXT\SAG_BONE_TORSO"
_dir_dicom2 = r"C:\Users\CH258598\Desktop\Current Research\case-100012\omi\incomingdir\case-100012\STANDARD_HEAD-NECK-U-EXT\SAG_ST_TORSO"
# Directory contatining multiple series
_dir_dicom3 = r"C:\Users\CH258598\Desktop\Current Research\case-100012\omi\incomingdir\case-100012\STANDARD_HEAD-NECK-U-EXT"
_dir_dicom3 = r"C:\Users\CH258598\Desktop\Current Research\PlayingData\case-100205\omi\incomingdir\case-100205\STANDARD_HEAD-NECK-U-EXT"
_dir_dicom4 = r"Z:\Object_Detection\Samples\Ct-Lumbar Spine W-O Contrast\0.6MM BONE - 4"

### Create container

In [4]:
# Create container
my_Container = SeriesContainer(verbose = True)

### Edit tags that will be loaded

In [5]:
# Set tags if necessary
print(f"Tags in my_Container: {my_Container.tag_list}")
my_Container.edit_tags([["StudyDate", "0008|0030"]])
print(f"Tags in my_Container: {my_Container.tag_list}")

Tags in my_Container: [['Modality', '0008|0060'], ['BodyPartExamined', '0018|0015'], ['SeriesDescription', '0008|103e'], ['SliceThickness', '0018|0050'], ['ImageOrientationPatient', '0020|0037']]
Tags in my_Container: [['Modality', '0008|0060'], ['BodyPartExamined', '0018|0015'], ['SeriesDescription', '0008|103e'], ['SliceThickness', '0018|0050'], ['ImageOrientationPatient', '0020|0037'], ['StudyDate', '0008|0030']]


### Load directory

In [6]:
# Load data
my_Container.obtain_data_from_series(path_dir = _dir_dicom4, mode = 'o')

# Print it
print(len(my_Container), my_Container.data)

# See all keys stored in for one series in container
print(my_Container.data[0].keys())

Found 1 potential series dirs in Z:\Object_Detection\Samples\Ct-Lumbar Spine W-O Contrast\0.6MM BONE - 4
--------------------------------------------------------------------------------------
In dir: Z:\Object_Detection\Samples\Ct-Lumbar Spine W-O Contrast\0.6MM BONE - 4 found 1 series.
Working on series 1/1
Total number of found series is: 1
--------------------------------------------------------------------------------------
1 [{'main_dir': 'Z:\\Object_Detection\\Samples\\Ct-Lumbar Spine W-O Contrast\\0.6MM BONE - 4', 'series_ID': '1.3.12.2.1107.5.1.4.75536.30000024012209561805700000190', 'series_tags': {'0008|0060': ['CT', 'Modality'], '0018|0015': ['SPINE ', 'BodyPartExamined'], '0008|103e': ['0.6MM BONE', 'SeriesDescription'], '0018|0050': ['0.6 ', 'SliceThickness'], '0020|0037': ['1\\0\\0\\0\\1\\0 ', 'ImageOrientationPatient'], '0008|0030': ['084551', 'StudyDate']}, 'series_image_info': {'size': (512, 512, 407), 'spacing': (0.26171875, 0.26171875, 0.30000000000000004), 'origin':

### Add new one

In [ ]:
# Add new series
my_Container.obtain_data_from_series(path_dir = _dir_dicom2, mode = 'a')

# Check if it is added
for _data in my_Container:
    print(_data['main_dir'])

### Add directory with all subdirectories

In [ ]:
# Load data mode override
my_Container.obtain_data_from_series(path_dir = _dir_dicom3, mode = 'o')

# Print it
print(f"Number of series in container: {len(my_Container)}")

### Reconstuct and preview image

In [7]:
# Reconstructed image
_image = reconstruct_image(my_Container.data[0], display = True)
print(type(_image))
_save_file = "test.nii.gz"
sitk.WriteImage(_image, _save_file)


<class 'SimpleITK.SimpleITK.Image'>


### Export container to directory

In [8]:
# Exporting to results
my_Container.export_data_to_directory("results", export_nifti = True, export_npy = True)

Directory results created.
--------------------------------------------------------------------------------------


IndexError: list index out of range

### Load data from exported dir

In [ ]:
my_Container.load_data_from_directory(load_dir_path = "results")
print(f"Number of series in container: {len(my_Container)}")

### Verify loaded data

In [ ]:
# Find same pair
_init_series_ID = my_Container.data[0]["series_ID"]

for _i, _item in enumerate(my_Container):
    if _i == 0:
        continue

    if _item["series_ID"] == _init_series_ID:
        reconstruct_image(_item, display = True)
        break

In [ ]:
import os
def obtain_series_dirs(path_dir: str = None)->list:
    """
    Method which seeks for the directories inside the main dir. It lists them
    and returns them. If there is no subdirectories then it is assumed that given
    directory is the one which contain series.

    Args:
        * path_dir, str, path to input dir in which we seek for series
    
    Return: list, list containing either given path (assuming that it contain series)
        or list of subdirectories in given dir assuming they represent series.
    """
    # Storage
    _directories_list = []

    # Find all directories inside the given directory which contain dcm file
    # inside
    for _root, _dirs, _files in os.walk(path_dir):
        if any(_file.endswith('.dcm') for _file in _files):
            _directories_list.append(_root)
    print(_directories_list)
    # Test
    if not _directories_list:
        print(f"Found 1 potential series dir in {path_dir}")
        return [path_dir]
    else:
        print(f"Found {len(_directories_list)} potential series dirs in {path_dir}")
        return _directories_list
_dir = r"C:\Users\CH258598\Desktop\Current Research\PlayingData\case-122285 - Copy"
_q = obtain_series_dirs(_dir)